In [1]:
import os
import matplotlib.pyplot as plt
import pims
import sys
import numpy as np
sys.path.insert(0,'/home/jmamede/scripts')
# from support_pla import othercolor, cell_mask, multiply, rebin
from support_pla import cellpose_mask,cellpose_model
import pandas as pd
from nd2reader import ND2Reader
import glob
import csv
plot = True
import napari
from napari.qt import thread_worker
from cellpose.utils import masks_to_outlines
from napari.qt import thread_worker
import sys, contextlib, io
import cellpose
sys.path.insert(0,'/home/jmamede/scripts')
from support_pla import cellpose_mask,cellpose_model
from cellpose import io as iocp
from cellpose import plot,utils
import matplotlib.pyplot as plt
import pims
import time
import numpy as np
from nd2reader import ND2Reader

In [2]:
# ficheiro = "/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA01_ChannelGFP,DAPI_Seq0000.nd2"
dirname = '/run/media/jmamede/Joao/96wells/20201119_143803_601/*.nd2'
filelist = glob.glob(dirname)
filelist.sort()


# reader = ND2Reader(ficheiro)
# print(reader.sizes)

# metadata = reader.metadata
# reader.bundle_axes = 'cyx'
# reader.iter_axes = 'z'
# #there is a bug, it reads each visit point as a Z.
# reader.default_coords['v'] = 0
# #reader.sizes['z']

# def plots(reader,cells_masks,nuclei_masks,base,series):
#     %matplotlib inline
#     from cellpose.utils import masks_to_outlines
#     f, axarr = plt.subplots(3)
#     print(axarr.shape)
#     axarr[0].imshow(reader[series][1],vmin=np.min(reader[5][1]),vmax=np.max(reader[5][1])/2
# #         ,cmap="gray"
#                      )
#     axarr[0].imshow(masks_to_outlines(cells_masks),alpha=0.6,cmap='gray'
#         )
# #     axarr[0,2].imshow(reader[i][0],vmin=100,vmax=np.max(reader[5][0])/3
# #         )
#     axarr[2].imshow(reader[series][1],vmin=np.min(reader[5][1]),vmax=np.max(reader[5][1])/2
# #         ,cmap="gray"
#                      )
#     axarr[2].imshow(cells_masks,alpha=0.4,cmap='jet')
    
#     axarr[1].imshow(reader[series][0],vmin=np.min(reader[5][0]),vmax=np.max(reader[5][0])/2
# #                       ,cmap="gray"
#         )
#     axarr[1].imshow(masks_to_outlines(nuclei_masks),alpha=0.4,cmap='gray')
        
        
# #     axarr[1,2].imshow(reader[i][1],vmin=100,vmax=np.max(reader[5][0])/3
# #         )
# #     axarr[1,2].imshow(nuclei_masks
# # #                 ,alpha=0.6
# #         )
#     plt.savefig(base+'v'+str(series),dpi=600)
#     return f,axarr



model = cellpose_model(GPU=True)
model_nuc = cellpose_model(GPU=True,model_type='nuclei')


** CUDA version installed and working. **
>>>> using GPU
cellpose_residual_on_style_on_concatenation_off
** CUDA version installed and working. **
>>>> using GPU
cellpose_residual_on_style_on_concatenation_off


In [3]:
filelist

['/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA01_ChannelGFP,DAPI_Seq0000.nd2',
 '/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA02_ChannelGFP,DAPI_Seq0001.nd2',
 '/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA03_ChannelGFP,DAPI_Seq0002.nd2',
 '/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA04_ChannelGFP,DAPI_Seq0003.nd2',
 '/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA05_ChannelGFP,DAPI_Seq0004.nd2',
 '/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA06_ChannelGFP,DAPI_Seq0005.nd2',
 '/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA07_ChannelGFP,DAPI_Seq0006.nd2',
 '/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA08_ChannelGFP,DAPI_Seq0007.nd2',
 '/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA09_ChannelGFP,DAPI_Seq0008.nd2',
 '/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA10_ChannelGFP,DAPI_Seq0009.nd2',
 '/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA11_ChannelGFP,DAPI_Se

In [4]:


sys.path.insert(0,'/home/jmamede/scripts')
# from support_pla import othercolor, cell_mask, multiply, rebin
from support_pla import cellpose_mask,cellpose_model

with napari.gui_qt():
    v = napari.Viewer(show=False)
    v.show()
    #     layer = viewer.add_image(np.random.random((512, 512)))
    def updating(result):
    #         v.layers.remove('image')
        from cellpose.utils import masks_to_outlines
        import time
        [v.layers.pop(0) for la in range(len(v.layers))]    

        v.add_image(
            result[1],
            blending='additive',
            contrast_limits=[np.min(result[1]),np.max(result[1])],
    #         ndisplay=2,
            colormap='blue',
            name='nuclei'
        )   

        v.add_image(
            result[0],
            blending='additive',
            colormap='green',
            name='cells')

        v.add_labels(
        result[2],
        opacity=0.4,
        blending='additive',
#             contrast_limits=[np.min(result[2]),np.max(result[2])],
        name='nuclei'
        ) 

        v.add_labels(
        masks_to_outlines(result[3]),
    #     utils.outlines_list(result[0]),
        opacity=0.4,
        blending='additive',
    #         contrast_limits=[np.min(result[1]),np.max(result[1])],
        name='nuclei'
        )
        
    @thread_worker(connect={'yielded': updating})
    def poseri(filelist,chan_cells,chan_nuclei,model,model_nuc):

        for ficheiro in filelist:
            reader = ND2Reader(ficheiro)
        #     metadata = reader.metadata
            reader.bundle_axes = 'cyx'
            reader.iter_axes = 'z'
            #there is a bug, it reads each visit point as a Z.
            reader.default_coords['v'] = 0
            ncells = 0
            nnuclei = 0

            for i in range(reader.sizes['z']):
    #         for i in [0]:
                #shut cellpose messages up.
    #             with contextlib.redirect_stdout(io.StringIO()):

                cells_masks = cellpose_mask(reader[i][chan_cells],model
                #                             ,size=2500
                                            ,flow_threshold=0.8,size=0,diam=50,cell_prob=-1
                                           )
                nuclei_masks = cellpose_mask(reader[i][chan_nuclei],model_nuc
                #                             ,size=2500
                                            ,flow_threshold=1.2,size=0,diam=25,cell_prob=0
                                           )
        #             ncells += cells_masks.max()
        #             nnuclei += nuclei_masks.max()
                base = os.path.splitext(ficheiro)[0]
                outlines = utils.outlines_list(cells_masks)
                cell_outlines = utils.masks_to_outlines(cells_masks)
                iocp.outlines_to_text(base+'v'+str(i), outlines)

        #             f,axarr =  plots(reader,cells_masks,nuclei_masks,base,i)

                text = "Image#: {0} Number of Nuclei: {1} Number of positive cells: {2} Percentage: {3}".format(i,nuclei_masks.max(),cells_masks.max(),cells_masks.max()/nuclei_masks.max())
                print(ficheiro,text)
        #         output.writerow([ficheiro,i,nuclei_masks.max(),cells_masks.max(),cells_masks.max()/nuclei_masks.max()*100])
        #     output2.writerow(([ficheiro,'NA',nnuclei,ncells,ncells/nnuclei*100]))
                yield [reader[i][chan_cells],reader[i][chan_nuclei],cells_masks,nuclei_masks]
    
        
#     from cellpose import io as iocp
#     from cellpose import plot,utils



    model = cellpose_model(GPU=True)
    model_nuc = cellpose_model(GPU=True,model_type='nuclei')

#     for ficheiro in filelist:
    poseri(filelist,0,1,model,model_nuc)
        
#         average_large_image()
#         result.returned.connect(v.add_image)
#         result.start()


#         worker.start

    #     layers[0].data = []
    #     layers[1].data = []
    #     layers[0] = []
    #     layers[0] = []

** CUDA version installed and working. **
>>>> using GPU
cellpose_residual_on_style_on_concatenation_off
** CUDA version installed and working. **
>>>> using GPU
cellpose_residual_on_style_on_concatenation_off


/home/jmamede/anaconda3/envs/pycuda/lib/python3.7/site-packages/nd2reader/raw_metadata.py:171: UserWarning: Z-levels details missing in metadata. Using Z-coordinates instead.
  warnings.warn("Z-levels details missing in metadata. Using Z-coordinates instead.")


processing 1 image(s)
time spent: running network 3.22s; flow+mask computation 0.15
estimated masks for 1 image(s) in 3.37 sec
>>>> TOTAL TIME 3.37 sec
processing 1 image(s)
time spent: running network 0.68s; flow+mask computation 0.49
estimated masks for 1 image(s) in 1.17 sec
>>>> TOTAL TIME 1.17 sec
/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA01_ChannelGFP,DAPI_Seq0000.nd2 Image#: 0 Number of Nuclei: 283 Number of positive cells: 21 Percentage: 0.07420494699646643
processing 1 image(s)
time spent: running network 0.85s; flow+mask computation 0.20
estimated masks for 1 image(s) in 1.07 sec
>>>> TOTAL TIME 1.07 sec
processing 1 image(s)
time spent: running network 0.62s; flow+mask computation 0.55
estimated masks for 1 image(s) in 1.17 sec
>>>> TOTAL TIME 1.17 sec
/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA01_ChannelGFP,DAPI_Seq0000.nd2 Image#: 1 Number of Nuclei: 269 Number of positive cells: 33 Percentage: 0.12267657992565056
processing 1 image(s)
time spe

/home/jmamede/anaconda3/envs/pycuda/lib/python3.7/site-packages/nd2reader/raw_metadata.py:171: UserWarning: Z-levels details missing in metadata. Using Z-coordinates instead.
  warnings.warn("Z-levels details missing in metadata. Using Z-coordinates instead.")


processing 1 image(s)
time spent: running network 0.83s; flow+mask computation 0.16
estimated masks for 1 image(s) in 1.01 sec
>>>> TOTAL TIME 1.01 sec
processing 1 image(s)
time spent: running network 0.63s; flow+mask computation 0.54
estimated masks for 1 image(s) in 1.18 sec
>>>> TOTAL TIME 1.18 sec
/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA02_ChannelGFP,DAPI_Seq0001.nd2 Image#: 0 Number of Nuclei: 351 Number of positive cells: 20 Percentage: 0.05698005698005698
processing 1 image(s)
time spent: running network 0.74s; flow+mask computation 0.20
estimated masks for 1 image(s) in 0.94 sec
>>>> TOTAL TIME 0.94 sec
processing 1 image(s)
time spent: running network 0.66s; flow+mask computation 0.46
estimated masks for 1 image(s) in 1.13 sec
>>>> TOTAL TIME 1.13 sec
/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA02_ChannelGFP,DAPI_Seq0001.nd2 Image#: 1 Number of Nuclei: 246 Number of positive cells: 25 Percentage: 0.1016260162601626
processing 1 image(s)
time spen

/home/jmamede/anaconda3/envs/pycuda/lib/python3.7/site-packages/nd2reader/raw_metadata.py:171: UserWarning: Z-levels details missing in metadata. Using Z-coordinates instead.
  warnings.warn("Z-levels details missing in metadata. Using Z-coordinates instead.")


processing 1 image(s)
time spent: running network 0.96s; flow+mask computation 0.12
estimated masks for 1 image(s) in 1.09 sec
>>>> TOTAL TIME 1.09 sec
processing 1 image(s)
time spent: running network 0.62s; flow+mask computation 0.53
estimated masks for 1 image(s) in 1.16 sec
>>>> TOTAL TIME 1.16 sec
/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA03_ChannelGFP,DAPI_Seq0002.nd2 Image#: 0 Number of Nuclei: 327 Number of positive cells: 12 Percentage: 0.03669724770642202
processing 1 image(s)
time spent: running network 0.75s; flow+mask computation 0.13
estimated masks for 1 image(s) in 0.90 sec
>>>> TOTAL TIME 0.90 sec
processing 1 image(s)
time spent: running network 0.64s; flow+mask computation 0.47
estimated masks for 1 image(s) in 1.12 sec
>>>> TOTAL TIME 1.12 sec
/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA03_ChannelGFP,DAPI_Seq0002.nd2 Image#: 1 Number of Nuclei: 283 Number of positive cells: 16 Percentage: 0.05653710247349823
processing 1 image(s)
time spe

/home/jmamede/anaconda3/envs/pycuda/lib/python3.7/site-packages/nd2reader/raw_metadata.py:171: UserWarning: Z-levels details missing in metadata. Using Z-coordinates instead.
  warnings.warn("Z-levels details missing in metadata. Using Z-coordinates instead.")


processing 1 image(s)
time spent: running network 0.76s; flow+mask computation 0.14
estimated masks for 1 image(s) in 0.90 sec
>>>> TOTAL TIME 0.90 sec
processing 1 image(s)
time spent: running network 0.61s; flow+mask computation 0.52
estimated masks for 1 image(s) in 1.13 sec
>>>> TOTAL TIME 1.13 sec
/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA04_ChannelGFP,DAPI_Seq0003.nd2 Image#: 0 Number of Nuclei: 256 Number of positive cells: 25 Percentage: 0.09765625
processing 1 image(s)
time spent: running network 0.88s; flow+mask computation 0.12
estimated masks for 1 image(s) in 1.02 sec
>>>> TOTAL TIME 1.02 sec
processing 1 image(s)
time spent: running network 0.65s; flow+mask computation 0.46
estimated masks for 1 image(s) in 1.12 sec
>>>> TOTAL TIME 1.12 sec
/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA04_ChannelGFP,DAPI_Seq0003.nd2 Image#: 1 Number of Nuclei: 261 Number of positive cells: 12 Percentage: 0.04597701149425287


In [7]:
import time
import numpy as np
import napari

from qtpy.QtCore import Qt
from qtpy.QtWidgets import QPushButton


image = np.random.random((2048, 2048))
zeros = np.zeros((2048, 2048))

@thread_worker
def pan(layer, interval=0.05):
    layer.translate = (-2148, 0)
    layer.data = image
    for i in range(-2140, 8, 16):
        layer.translate = (i, 0)
        yield
        time.sleep(interval)

with napari.gui_qt():
    v = napari.Viewer(show=False)
    v.window._qt_window.setWindowState(Qt.WindowMaximized)
    v.show()

    image_layer = v.add_image(zeros)
    worker = pan(image_layer)

    button = QPushButton("Start animation")
    button.clicked.connect(worker.start)
    v.window.add_dock_widget(widget=button, area='left')

NameError: name 'masks_to_outlines' is not defined

In [30]:
with napari.gui_qt():
    viewer = napari.Viewer()

    @thread_worker(connect={"returned": viewer.add_image})
    def average_large_image():
        return np.random.rand(1024, 512, 512).mean(0)

    average_large_image()

In [7]:
import napari
with napari.gui_qt():
    v = napari.view_image(result[0])

/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/__init__.py:44: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.7. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

In [ ]:
from cellpose.utils import 

plt.imshow(masks_to_outlines(cells_masks))